Copyright **`(c)`** 2024 Federico Spinoso `<s324617@studenti.polito.it>`  
[`https://github.com/fedspi00`](https://github.com/fedspi00)  

## Lab 3 - Path search Gem Puzzle

In [177]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [178]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [179]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


In [180]:
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

One of the most common heuristics is the Manhattan distance, a function that estimates the cost to reach the goal from the current state.

In [181]:
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    """Calculate the Manhattan distance between the current state and the goal."""
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0:
                continue
            target_x, target_y = divmod(value - 1, PUZZLE_DIM)
            distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [182]:
def is_goal(state: np.ndarray, goal: np.ndarray) -> bool:
    """Check if the current state is the goal state."""
    return np.array_equal(state, goal)

### **A\* Algorithm** 
- A\* combines BFS and heuristics to explore nodes with the lowest estimated cost \(f(n) = g(n) + h(n)\).
- The heuristic \(h(n)\) (e.g., Manhattan distance) estimates the cost to reach the goal.
- A\* is optimal when the heuristic is admissible (does not overestimate the true cost).

In [183]:
def a_star_solve(start_state: np.ndarray, goal_state: np.ndarray) -> list[np.ndarray]:
    """Solve the n^2-1 puzzle using the A* algorithm."""
    frontier = [(0, tuple(start_state.flatten()), start_state)]
    
    came_from = {tuple(start_state.flatten()): None}
    cost_so_far = {tuple(start_state.flatten()): 0}
    
    while frontier:
        _, current_tuple, current_state = frontier.pop(0)
        
        if is_goal(current_state, goal_state):
            path = []
            while current_tuple is not None:
                path.append(current_state)
                current_tuple = came_from[current_tuple]
                current_state = None if current_tuple is None else np.array(current_tuple).reshape(PUZZLE_DIM, PUZZLE_DIM)
            return path[::-1] 
        
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_tuple = tuple(next_state.flatten())
            new_cost = cost_so_far[current_tuple] + 1  
            
            if next_tuple not in cost_so_far or new_cost < cost_so_far[next_tuple]:
                cost_so_far[next_tuple] = new_cost
                priority = new_cost + manhattan_distance(next_state, goal_state)
                
                inserted = False
                for i in range(len(frontier)):
                    if frontier[i][0] > priority:
                        frontier.insert(i, (priority, next_tuple, next_state))
                        inserted = True
                        break
                if not inserted:
                    frontier.append((priority, next_tuple, next_state))
                
                came_from[next_tuple] = current_tuple

    return None  




### **Iterative Deepening Depth-First Search (IDDFS)**

- IDDFS is an improved version of the DFS and explores as far down a branch as possible before backtracking until a solution is found or the maximum depth is reached.
- However, it does not guarantee the shortest solution.

In [184]:
def iddfs_solve(start_state: np.ndarray, goal_state: np.ndarray, max_depth=30) -> list[np.ndarray]:
    """Solve the n^2-1 puzzle using Iterative Deepening DFS (IDDFS)."""
    start_tuple = tuple(start_state.flatten())
    goal_tuple = tuple(goal_state.flatten())

    def dfs(current_tuple, path, depth, visited):
        if depth == 0:
            return None
        if current_tuple == goal_tuple:
            return path + [current_tuple]
        visited.add(current_tuple)
        current_state = np.array(current_tuple).reshape(start_state.shape)
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_tuple = tuple(next_state.flatten())
            if next_tuple not in visited:
                result = dfs(next_tuple, path + [current_tuple], depth - 1, visited)
                if result:
                    return result
        visited.remove(current_tuple) 
        return None

    for depth in range(1, max_depth + 1):
        visited = set()
        result = dfs(start_tuple, [], depth, visited)
        if result:
            return [np.array(p).reshape(start_state.shape) for p in result]

    return None 


### **Breadth-First Search (BFS)**

- BFS explores all nodes at the current depth level before moving deeper.
- It guarantees finding the shortest solution if one exists.

In [185]:
def bfs_solve(start_state: np.ndarray, goal_state: np.ndarray) -> list[np.ndarray]:
    """Solve the n^2-1 puzzle using BFS."""
    
    queue = [(start_state, [])]  
    visited = set() 

    while queue:
        current_state, path = queue.pop(0)

        if is_goal(current_state, goal_state):
            return path + [current_state]

        current_tuple = tuple(current_state.flatten())
        if current_tuple in visited:
            continue
        visited.add(current_tuple)

        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_tuple = tuple(next_state.flatten())

            if next_tuple not in visited:
                queue.append((next_state, path + [current_state]))

    return None 



In [186]:
# Generate a randomized start state
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

# Define the goal state
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

In [187]:
# Solve the puzzle using the A* approach
solution = a_star_solve(state, goal_state)

# Print the solution
if solution:
    print(f"Solution found in {len(solution) - 1} moves:")
    for step, s in enumerate(solution):
        print(f"Step {step}:\n{s}\n")
else:
    print("No solution found.")

Solution found in 24 moves:
Step 0:
[[4 5 1]
 [6 3 2]
 [8 7 0]]

Step 1:
[[4 5 1]
 [6 3 2]
 [8 0 7]]

Step 2:
[[4 5 1]
 [6 3 2]
 [0 8 7]]

Step 3:
[[4 5 1]
 [0 3 2]
 [6 8 7]]

Step 4:
[[0 5 1]
 [4 3 2]
 [6 8 7]]

Step 5:
[[5 0 1]
 [4 3 2]
 [6 8 7]]

Step 6:
[[5 1 0]
 [4 3 2]
 [6 8 7]]

Step 7:
[[5 1 2]
 [4 3 0]
 [6 8 7]]

Step 8:
[[5 1 2]
 [4 0 3]
 [6 8 7]]

Step 9:
[[5 1 2]
 [4 8 3]
 [6 0 7]]

Step 10:
[[5 1 2]
 [4 8 3]
 [0 6 7]]

Step 11:
[[5 1 2]
 [0 8 3]
 [4 6 7]]

Step 12:
[[0 1 2]
 [5 8 3]
 [4 6 7]]

Step 13:
[[1 0 2]
 [5 8 3]
 [4 6 7]]

Step 14:
[[1 2 0]
 [5 8 3]
 [4 6 7]]

Step 15:
[[1 2 3]
 [5 8 0]
 [4 6 7]]

Step 16:
[[1 2 3]
 [5 0 8]
 [4 6 7]]

Step 17:
[[1 2 3]
 [5 6 8]
 [4 0 7]]

Step 18:
[[1 2 3]
 [5 6 8]
 [4 7 0]]

Step 19:
[[1 2 3]
 [5 6 0]
 [4 7 8]]

Step 20:
[[1 2 3]
 [5 0 6]
 [4 7 8]]

Step 21:
[[1 2 3]
 [0 5 6]
 [4 7 8]]

Step 22:
[[1 2 3]
 [4 5 6]
 [0 7 8]]

Step 23:
[[1 2 3]
 [4 5 6]
 [7 0 8]]

Step 24:
[[1 2 3]
 [4 5 6]
 [7 8 0]]



In [188]:
# Solve the puzzle using the BFS approach
solution = bfs_solve(state, goal_state)

# Print the solution
if solution:
    print(f"Solution found in {len(solution) - 1} moves:")
    for step, s in enumerate(solution):
        print(f"Step {step}:\n{s}\n")
else:
    print("No solution found.")

Solution found in 24 moves:
Step 0:
[[4 5 1]
 [6 3 2]
 [8 7 0]]

Step 1:
[[4 5 1]
 [6 3 0]
 [8 7 2]]

Step 2:
[[4 5 1]
 [6 0 3]
 [8 7 2]]

Step 3:
[[4 0 1]
 [6 5 3]
 [8 7 2]]

Step 4:
[[4 1 0]
 [6 5 3]
 [8 7 2]]

Step 5:
[[4 1 3]
 [6 5 0]
 [8 7 2]]

Step 6:
[[4 1 3]
 [6 0 5]
 [8 7 2]]

Step 7:
[[4 1 3]
 [0 6 5]
 [8 7 2]]

Step 8:
[[4 1 3]
 [8 6 5]
 [0 7 2]]

Step 9:
[[4 1 3]
 [8 6 5]
 [7 0 2]]

Step 10:
[[4 1 3]
 [8 0 5]
 [7 6 2]]

Step 11:
[[4 1 3]
 [0 8 5]
 [7 6 2]]

Step 12:
[[0 1 3]
 [4 8 5]
 [7 6 2]]

Step 13:
[[1 0 3]
 [4 8 5]
 [7 6 2]]

Step 14:
[[1 3 0]
 [4 8 5]
 [7 6 2]]

Step 15:
[[1 3 5]
 [4 8 0]
 [7 6 2]]

Step 16:
[[1 3 5]
 [4 8 2]
 [7 6 0]]

Step 17:
[[1 3 5]
 [4 8 2]
 [7 0 6]]

Step 18:
[[1 3 5]
 [4 0 2]
 [7 8 6]]

Step 19:
[[1 3 5]
 [4 2 0]
 [7 8 6]]

Step 20:
[[1 3 0]
 [4 2 5]
 [7 8 6]]

Step 21:
[[1 0 3]
 [4 2 5]
 [7 8 6]]

Step 22:
[[1 2 3]
 [4 0 5]
 [7 8 6]]

Step 23:
[[1 2 3]
 [4 5 0]
 [7 8 6]]

Step 24:
[[1 2 3]
 [4 5 6]
 [7 8 0]]



In [189]:
# Solve the puzzle using the DFS approach
solution = iddfs_solve(state, goal_state)

# Print the solution
if solution:
    print(f"Solution found in {len(solution) - 1} moves:")
    for step, s in enumerate(solution):
        print(f"Step {step}:\n{s}\n")
else:
    print("No solution found.")

Solution found in 24 moves:
Step 0:
[[4 5 1]
 [6 3 2]
 [8 7 0]]

Step 1:
[[4 5 1]
 [6 3 0]
 [8 7 2]]

Step 2:
[[4 5 1]
 [6 0 3]
 [8 7 2]]

Step 3:
[[4 0 1]
 [6 5 3]
 [8 7 2]]

Step 4:
[[4 1 0]
 [6 5 3]
 [8 7 2]]

Step 5:
[[4 1 3]
 [6 5 0]
 [8 7 2]]

Step 6:
[[4 1 3]
 [6 0 5]
 [8 7 2]]

Step 7:
[[4 1 3]
 [0 6 5]
 [8 7 2]]

Step 8:
[[4 1 3]
 [8 6 5]
 [0 7 2]]

Step 9:
[[4 1 3]
 [8 6 5]
 [7 0 2]]

Step 10:
[[4 1 3]
 [8 0 5]
 [7 6 2]]

Step 11:
[[4 1 3]
 [0 8 5]
 [7 6 2]]

Step 12:
[[0 1 3]
 [4 8 5]
 [7 6 2]]

Step 13:
[[1 0 3]
 [4 8 5]
 [7 6 2]]

Step 14:
[[1 3 0]
 [4 8 5]
 [7 6 2]]

Step 15:
[[1 3 5]
 [4 8 0]
 [7 6 2]]

Step 16:
[[1 3 5]
 [4 8 2]
 [7 6 0]]

Step 17:
[[1 3 5]
 [4 8 2]
 [7 0 6]]

Step 18:
[[1 3 5]
 [4 0 2]
 [7 8 6]]

Step 19:
[[1 3 5]
 [4 2 0]
 [7 8 6]]

Step 20:
[[1 3 0]
 [4 2 5]
 [7 8 6]]

Step 21:
[[1 0 3]
 [4 2 5]
 [7 8 6]]

Step 22:
[[1 2 3]
 [4 0 5]
 [7 8 6]]

Step 23:
[[1 2 3]
 [4 5 0]
 [7 8 6]]

Step 24:
[[1 2 3]
 [4 5 6]
 [7 8 0]]

